In [3]:
# Importing the libraries
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout, GRU
from keras.optimizers import SGD
import math
from sklearn.metrics import mean_squared_error


In [4]:
pip install scikit-learn

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\DELL 3510\AppData\Local\Programs\Python\Python38\python.exe -m pip install --upgrade pip' command.


In [5]:
pip install gensim

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\DELL 3510\AppData\Local\Programs\Python\Python38\python.exe -m pip install --upgrade pip' command.


In [6]:
pip install spacy


  Using cached spacy-3.8.2.tar.gz (1.3 MB)
  Installing build dependencies: started
  Installing build dependencies: still running...
  Installing build dependencies: still running...
  Installing build dependencies: still running...
  Installing build dependencies: finished with status 'error'
  Using cached spacy-3.7.5-cp38-cp38-win_amd64.whl (12.5 MB)
  Using cached pydantic-2.10.6-py3-none-any.whl (431 kB)
  Using cached wasabi-1.1.3-py3-none-any.whl (27 kB)
  Using cached spacy_loggers-1.0.5-py3-none-any.whl (22 kB)
  Using cached catalogue-2.0.10-py3-none-any.whl (17 kB)
  Using cached weasel-0.4.1-py3-none-any.whl (50 kB)
  Using cached thinc-8.2.5.tar.gz (193 kB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'error'
  Using cached thinc-8.2.4-cp38-cp38-win_amd64.whl (1.5 MB)
  Using cached murmurhash-1.0.12.tar.gz (13 kB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Gett

  ERROR: Command errored out with exit status 1:
   command: 'c:\Users\DELL 3510\AppData\Local\Programs\Python\Python38\python.exe' 'C:\Users\DELL 3510\AppData\Local\Temp\pip-standalone-pip-lairx538\__env_pip__.zip\pip' install --ignore-installed --no-user --prefix 'C:\Users\DELL 3510\AppData\Local\Temp\pip-build-env-yob3oxxj\overlay' --no-warn-script-location --no-binary :none: --only-binary :none: -i https://pypi.org/simple -- setuptools 'cython>=0.25,<3.0' 'cymem>=2.0.2,<2.1.0' 'preshed>=3.0.2,<3.1.0' 'murmurhash>=0.28.0,<1.1.0' 'thinc>=8.3.0,<8.4.0' 'numpy>=2.0.0,<2.1.0; python_version < '"'"'3.9'"'"'' 'numpy>=2.0.0,<2.1.0; python_version >= '"'"'3.9'"'"''
       cwd: None
  Complete output (293 lines):
  Ignoring numpy: markers 'python_version >= "3.9"' don't match your environment
    Using cached setuptools-75.3.0-py3-none-any.whl (1.3 MB)
    Using cached Cython-0.29.37-py2.py3-none-any.whl (989 kB)
    Using cached cymem-2.0.11.tar.gz (10 kB)
    Installing build dependencies:

In [7]:
import pandas as pd

# Lire le fichier texte en précisant le séparateur (',' ici, à adapter selon ton fichier)
df = pd.read_csv("household_power_consumption.txt")  # Utilise '\t' si c'est une tabulation

# Sauvegarder en CSV
df.to_csv("household_power_consumption.csv", index=False)

print("Conversion terminée : household_power_consumption.csv est créé !")

Conversion terminée : household_power_consumption.csv est créé !


In [8]:
df = pd.read_csv("household_power_consumption.csv", sep=";")  # Now columns are separated properly
df.head()

C:\Users\DELL 3510\AppData\Local\Temp\ipykernel_12308\3098965311.py:1: DtypeWarning: Columns (2,3,4,5,6,7) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("household_power_consumption.csv", sep=";")  # Now columns are separated properly


,Date,Time,Global_active_power,Global_reactive_power,Voltage,Global_intensity,Sub_metering_1,Sub_metering_2,Sub_metering_3
0,16/12/2006,17:24:00,4.216,0.418,234.840,18.400,0.000,1.000,17.0
1,16/12/2006,17:25:00,5.360,0.436,233.630,23.000,0.000,1.000,16.0
2,16/12/2006,17:26:00,5.374,0.498,233.290,23.000,0.000,2.000,17.0
3,16/12/2006,17:27:00,5.388,0.502,233.740,23.000,0.000,1.000,17.0
4,16/12/2006,17:28:00,3.666,0.528,235.680,15.800,0.000,1.000,17.0


In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2075259 entries, 0 to 2075258
Data columns (total 9 columns):
 #   Column                 Dtype  
---  ------                 -----  
 0   Date                   object 
 1   Time                   object 
 2   Global_active_power    object 
 3   Global_reactive_power  object 
 4   Voltage                object 
 5   Global_intensity       object 
 6   Sub_metering_1         object 
 7   Sub_metering_2         object 
 8   Sub_metering_3         float64
dtypes: float64(1), object(8)
memory usage: 142.5+ MB


In [10]:
import pandas as pd

# Charger les données
df['Datetime'] = pd.to_datetime(df['Date'] + ' ' + df['Time'])
df = df.drop(columns=['Date', 'Time'])  # Supprimer les colonnes inutiles

# Conversion des colonnes numériques
cols_to_convert = ['Global_active_power', 'Global_reactive_power', 'Voltage',
                   'Global_intensity', 'Sub_metering_1', 'Sub_metering_2']

df[cols_to_convert] = df[cols_to_convert].apply(pd.to_numeric, errors='coerce')

# Convertir les colonnes en float (sans toucher à "Datetime")
df[cols_to_convert] = df[cols_to_convert].astype(float)

# Créer la colonne cible (valeur du lendemain)
df['Target'] = df['Global_active_power'].shift(-1)

# Supprimer les lignes avec valeurs manquantes
df = df.dropna()

# Vérification
print(df.dtypes)
print(df.head())


C:\Users\DELL 3510\AppData\Local\Temp\ipykernel_12308\1318260057.py:4: UserWarning: Parsing dates in %d/%m/%Y %H:%M:%S format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df['Datetime'] = pd.to_datetime(df['Date'] + ' ' + df['Time'])


Global_active_power             float64
Global_reactive_power           float64
Voltage                         float64
Global_intensity                float64
Sub_metering_1                  float64
Sub_metering_2                  float64
Sub_metering_3                  float64
Datetime                 datetime64[ns]
Target                          float64
dtype: object
   Global_active_power  Global_reactive_power  Voltage  Global_intensity  \
0                4.216                  0.418   234.84              18.4   
1                5.360                  0.436   233.63              23.0   
2                5.374                  0.498   233.29              23.0   
3                5.388                  0.502   233.74              23.0   
4                3.666                  0.528   235.68              15.8   

   Sub_metering_1  Sub_metering_2  Sub_metering_3            Datetime  Target  
0             0.0             1.0            17.0 2006-12-16 17:24:00   5.360  
1        

In [11]:
# Séparation des caractéristiques (X) et de la cible (y)
X = df.drop(columns=['Target', 'Datetime'])  # Supprimer la cible et la date pour l'entraînement
y = df['Target']

# Normalisation des caractéristiques
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)
# Conversion en DataFrame
df_scaled = pd.DataFrame(X_scaled, columns=X.columns)
df_scaled['Target'] = y.values  # Ajouter la cible non normalisée

# Vérifier le résultat
print(df_scaled.head())
print(df_scaled.dtypes)


   Global_active_power  Global_reactive_power   Voltage  Global_intensity  \
0             0.374796               0.300719  0.376090          0.377593   
1             0.478363               0.313669  0.336995          0.473029   
2             0.479631               0.358273  0.326010          0.473029   
3             0.480898               0.361151  0.340549          0.473029   
4             0.325005               0.379856  0.403231          0.323651   

   Sub_metering_1  Sub_metering_2  Sub_metering_3  Target  
0             0.0          0.0125        0.548387   5.360  
1             0.0          0.0125        0.516129   5.374  
2             0.0          0.0250        0.548387   5.388  
3             0.0          0.0125        0.548387   3.666  
4             0.0          0.0125        0.548387   3.520  
Global_active_power      float64
Global_reactive_power    float64
Voltage                  float64
Global_intensity         float64
Sub_metering_1           float64
Sub_metering

In [12]:
pip install torch

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\DELL 3510\AppData\Local\Programs\Python\Python38\python.exe -m pip install --upgrade pip' command.


In [13]:
def create_sequences(data, seq_length):
    X, y = [], []
    for i in range(len(data) - seq_length):
        X.append(data.iloc[i:i + seq_length].values)  # Séquence d'entrée (fenêtre glissante)
        y.append(data.iloc[i + seq_length]['Global_active_power'])  # Valeur cible (ici on suppose que l'on prédit la puissance active)
    return np.array(X), np.array(y)

# Exemple de création de séquences
seq_length = 24  # 24 heures
X, y = create_sequences(df_scaled, seq_length)

# Division en ensembles d'entraînement, validation et test
from sklearn.model_selection import train_test_split

X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.2, shuffle=False)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, shuffle=False)

# Vérification des tailles
print(f"Train: {X_train.shape}, Val: {X_val.shape}, Test: {X_test.shape}")


Train: (1639347, 24, 8), Val: (204918, 24, 8), Test: (204919, 24, 8)


In [14]:
import torch
from torch.utils.data import Dataset, DataLoader

class TimeSeriesDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.float32)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

# Création des datasets
train_dataset = TimeSeriesDataset(X_train, y_train)
val_dataset = TimeSeriesDataset(X_val, y_val)
test_dataset = TimeSeriesDataset(X_test, y_test)

# Utilisation de DataLoader pour le chargement des données
batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Vérification des tailles des batches
for X_batch, y_batch in train_loader:
    print(f"Batch X: {X_batch.shape}, Batch y: {y_batch.shape}")
    break

Batch X: torch.Size([32, 24, 8]), Batch y: torch.Size([32])


In [16]:
import torch
import torch.nn as nn
import torch.optim as optim


In [17]:
class LSTM_GRU_Model(nn.Module):
    def __init__(self, input_size, hidden_size, seq_length, output_size):
        super(LSTM_GRU_Model, self).__init__()
        
        # Première couche GRU
        self.gru1 = nn.GRU(input_size=input_size, hidden_size=hidden_size, batch_first=True, dropout=0.2)
        
        # Deuxième couche LSTM
        self.lstm = nn.LSTM(input_size=hidden_size, hidden_size=hidden_size, batch_first=True, dropout=0.2)
        
        # Couche de Dropout pour éviter le sur-ajustement
        self.dropout = nn.Dropout(0.3)
        
        # Couche Dense (fully connected)
        self.fc = nn.Linear(hidden_size, output_size)
        
    def forward(self, x):
        # Passage par la couche GRU
        gru_out, _ = self.gru1(x)
        
        # Passage par la couche LSTM
        lstm_out, _ = self.lstm(gru_out)
        
        # On récupère la dernière sortie
        last_out = lstm_out[:, -1, :]
        
        # Passage par la couche Dense
        output = self.fc(self.dropout(last_out))
        
        return output

In [18]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import torch
from torch.utils.data import DataLoader, TensorDataset

# Extraire les caractéristiques temporelles
df['year'] = df['Datetime'].dt.year
df['month'] = df['Datetime'].dt.month
df['day'] = df['Datetime'].dt.day
df['hour'] = df['Datetime'].dt.hour
df['minute'] = df['Datetime'].dt.minute
df['second'] = df['Datetime'].dt.second

# Sélectionner les colonnes pertinentes
features = ['Global_active_power', 'Global_reactive_power', 'Voltage', 'Global_intensity', 
            'Sub_metering_1', 'Sub_metering_2', 'Sub_metering_3', 'year', 'month', 'day', 'hour', 'minute', 'second']
target = 'Target'

# Préparer les données d'entrée et la cible
X = df[features].values
y = df[target].values

# Normalisation des données (mise à l'échelle)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Conversion en tenseurs PyTorch
X_tensor = torch.tensor(X_scaled, dtype=torch.float32)
y_tensor = torch.tensor(y, dtype=torch.float32).view(-1, 1)

# Diviser les données en ensembles d'entraînement et de validation
X_train, X_val, y_train, y_val = train_test_split(X_tensor, y_tensor, test_size=0.2, shuffle=False)

# Créer des DataLoader pour l'entraînement et la validation
train_dataset = TensorDataset(X_train, y_train)
val_dataset = TensorDataset(X_val, y_val)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

# Afficher la taille des ensembles d'entraînement et de validation
print(f"Taille d'entraînement: {len(train_loader.dataset)} échantillons")
print(f"Taille de validation: {len(val_loader.dataset)} échantillons")


Taille d'entraînement: 1639366 échantillons
Taille de validation: 409842 échantillons
